# Machine learning report - Credit card fraud detection

In [216]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder

training_file_path = 'training.csv'
X_train = pd.read_csv(training_file_path)

test_file_path = 'test.csv'
X_test = pd.read_csv(test_file_path)

## 1. Préparation des données
### 1.1 Exploration des données
Nous allons commencer par explorer les données afin de voir si elles sont exploitables et si elles nécessitent un traitement particulier.

Les points d'attention sont les suivants :
- Les données contiennent-elles des valeurs manquantes ?
- Les données contiennent-elles des variables catégorielles ?
- Les données sont elles équilibrées ?
- Les données sont elles au format numérique ?

In [217]:
# On print le nombre de valeurs manquantes par colonne
print(f"Valeurs manquantes: {X_train.isnull().sum()}")

# On mesure l'équilibre des classes
X_train_no_fraud = X_train[X_train['FraudResult']==0]
X_train_fraud = X_train[X_train['FraudResult']==1]

percentage_minority = len(X_train_fraud)/(len(X_train_no_fraud) + len(X_train_fraud)) * 100
print(f"Pourcentage de fraude: {round(percentage_minority, 2)}%")

# On print le type de chaque colonne
print(f"Types:{X_train.dtypes}")

# On print le nombre de valeurs uniques par colonne
print(f"Valeurs uniques: {X_train.nunique()}")

# On print les premières lignes du dataset
print(f"Premières lignes: {X_train.head()}")

# On print les statistiques descriptives du dataset
print(f"Statistiques descriptives: {X_train.describe()}")

Valeurs manquantes: TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
dtype: int64
Pourcentage de fraude: 0.2%
Types:TransactionId            object
BatchId                  object
AccountId                object
SubscriptionId           object
CustomerId               object
CurrencyCode             object
CountryCode               int64
ProviderId               object
ProductId                object
ProductCategory          object
ChannelId                object
Amount                  float64
Value                     int64
TransactionStartTime     object
PricingStrategy           int64
FraudResult               int64
dtype

Voici les observations que nous avons faites sur les données :
- Les données ne contiennent pas de valeurs manquantes
- Les données sont fort déséquilibrées (0,2% de fraudes)
- Des données qui devraient être numériques sont au format object
- **CurrencyCode** et **CountryCode** ne sont pas exploitables car elles ne contienent qu'une seule valeur
- **ProviderId**, **ProductId**, **ProductCategory**, **ChannelId** et **PricingStrategy** sont des variables catégorielles

### 1.2 Rééquilibrage des données
Face a un jeu de données en déséquilibre, deux solutions s'offrent à nous:
- Effectuer un oversampling de la classe minoritaire (fraud) en créant des observations de cette classe.
- Effectuer un undersampling de la classe majoritaire (normal) en supprimant des observations de cette classe.

Nous allons ici choisir la première solution, en utilisant la methode resample de la librairie sklearn. Celle ci va nous permettre de créer un dataset équilibré en augmentant le nombre d'observations de la classe minoritaire.

In [218]:
X_train_fraud_upsampled = resample(X_train_fraud, replace=True, n_samples=len(X_train_no_fraud), random_state=123)

X_train_upsampled = pd.concat([X_train_no_fraud, X_train_fraud])

y = X_train_upsampled['FraudResult']
# On supprime la colonne FraudResult du dataset qui ne nous est désormais plus utile
X_train_upsampled = X_train_upsampled.drop(['FraudResult'], axis=1)

### 1.3 Selection des variables
Nous allons ici sélectionner les variables qui nous semblent pertinentes pour la prédiction. Nous allons donc retirer les variables suivantes :
- **CurrencyCode** et **CountryCode** car elles ne contiennent qu'une seule valeur
- **BatchId** car il s'agit de l'identifiant d'un groupe de transactions, non disponible au moment de la transaction et a priori pas lié à la fraude
- **TransactionId** car il s'agit d'un identifiant unique, a priori non lié à la fraude

In [219]:
# On supprime les colonnes suivantes
X_train_upsampled = X_train_upsampled.drop(['TransactionId', 'BatchId', 'CurrencyCode', 'CountryCode', 'TransactionStartTime'], axis=1)
# On stock les index (TransactionId) des lignes du dataset de test avant de supprimer la colonne TransactionId
X_test_index = X_test['TransactionId']
X_test = X_test.drop(['TransactionId', 'BatchId', 'CurrencyCode', 'CountryCode', 'TransactionStartTime'], axis=1)

### 1.4 Conversion des variables numériques
Nous allons convertir les variables numériques au format object en variables numériques au format int. Pour cela nous retirons la partie texte des valeurs et nous convertissons le résultat en int.

In [220]:
# On récupère les colonnes qui contiennent Id dans leur nom
id_columns = [col for col in X_train_upsampled.columns if 'Id' in col]

# On définit une fonction qui va enlever la partie textuelle des identifiants
def transform_id(feature):
    if isinstance(feature, str):
        return feature.split("_")[-1]
    else:
        return feature
    
# On applique la fonction sur les colonnes d'identifiants
for column in id_columns:
    X_train_upsampled[column] = X_train_upsampled[column].apply(transform_id)
    X_test[column] = X_test[column].apply(transform_id)

# On transforme les colonnes d'identifiants en int
X_train_upsampled[id_columns] = X_train_upsampled[id_columns].astype('int64')
X_test[id_columns] = X_test[id_columns].astype('int64')

### 1.5 Encodage des variables catégorielles
Nous allons encoder les variables catégorielles de deux manières différentes :
- **One hot encoding** pour les variables contenants moins de 10 modalités
- **Target encoding** pour les variables contenant plus de 10 modalités

On fera donc du **one hot encoding** sur les variables **ProviderId**, **ProductCategory**, **ChannelId** et **PricingStrategy**.
**ProductId** contient 23 modalités, nous allons donc faire du **target encoding** sur cette variable. Il est cependant déjà au format int, nous n'avons donc pas besoin de le convertir.

In [221]:
# On liste les colonnes catégorielles
categorical_columns = ['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'PricingStrategy']
# On ne garde que celles contenants moins de 10 modalités
categorical_columns = [col for col in categorical_columns if X_train_upsampled[col].nunique() < 10]

# On effectue un one hot encoding sur les colonnes catégorielles
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train_upsampled[categorical_columns]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[categorical_columns]))

# On remet les index
OH_cols_train.index = X_train_upsampled.index
OH_cols_test.index = X_test.index

# On enlève les colonnes catégorielles
num_X_train = X_train_upsampled.drop(categorical_columns, axis=1)
num_X_test = X_test.drop(categorical_columns, axis=1)

# On ajoute les colonnes encodées
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

# On transforme les noms de colonnes en string
OH_X_train.columns = OH_X_train.columns.map(str)
OH_X_test.columns = OH_X_test.columns.map(str)

# On print les premières lignes du dataset
print(OH_X_train.head())

   AccountId  SubscriptionId  CustomerId  ProductId   Amount  Value    0    1  \
0       3957             887        4406         10   1000.0   1000  0.0  0.0   
1       4841            3829        4406          6    -20.0     20  0.0  0.0   
2       4229             222        4683          1    500.0    500  0.0  0.0   
3        648            2185         988         21  20000.0  21800  1.0  0.0   
4       4841            3829         988          6   -644.0    644  0.0  0.0   

     2    3  ...   13   14   15   16   17   18   19   20   21   22  
0  0.0  0.0  ...  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  
1  0.0  1.0  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  
2  0.0  0.0  ...  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  
3  0.0  0.0  ...  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  
4  0.0  1.0  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  

[5 rows x 29 columns]


c:\Users\qjado\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## 2. Modélisation
### 2.1. Séparation des données
Nous allons ici séparer nos données en deux jeux de données:
- Un jeu de données d'entrainement (80% des données)
- Un jeu de données de test (20% des données)
Ceci nous permettra de tester la performance de notre modèle sur des données qu'il n'a jamais vu.
### 2.2 Choix du modèle
Nous allons ici utiliser un modèle de type **Random Forest**. Ce modèle est un modèle d'apprentissage supervisé qui peut être utilisé pour la classification ou la régression. Il s'agit d'un modèle très utilisé en machine learning car il est très performant et qu'il permet de traiter des données manquantes ou des données non équilibrées.
### 2.3. Entrainement du modèle
Nous allons ici entrainer notre modèle sur notre jeu de données d'entrainement. Nous allons ensuite tester la performance de notre modèle sur notre jeu de données de test.
### 2.4. Evaluation du modèle
Nous allons ici évaluer la performance de notre modèle en utilisant la matrice de confusion. Cette matrice nous permet de visualiser les prédictions de notre modèle. Elle nous permet de voir les vrais positifs, les vrais négatifs, les faux positifs et les faux négatifs. Nous allons également calculer le score de précision de notre modèle. Ce score nous permet de voir la proportion de prédictions correctes effectuées par notre modèle.
Nous calculons également le score f1 de notre modèle. Ce score est la moyenne harmonique entre la précision et le rappel. Il permet de mesurer la performance d'un modèle de classification binaire.


In [224]:
# Séparation des données en train et validation
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(OH_X_train, y, train_size=0.8, test_size=0.2, random_state=0)

# On définit le modèle
card_fraud_model = RandomForestClassifier(n_estimators=100, random_state=0)
card_fraud_model_XGB = XGBClassifier(n_estimators=2, random_state=0, learning_rate=0.1, max_depth=2, use_label_encoder=False)

# On entraine le modèle
card_fraud_model.fit(X_train, y_train)
card_fraud_model_XGB.fit(X_train, y_train)

# On fait des prédictions sur le dataset de validation
predictions = card_fraud_model.predict(X_valid)

# On evalue le modèle avec la métrique F1 et la matrice de confusion
from sklearn.metrics import f1_score, confusion_matrix

print(f1_score(y_valid, predictions))
print(confusion_matrix(y_valid, predictions))

# # On optimise les hyperparamètres du modèle XGBClassifier
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'n_estimators': [2, 4, 6, 8, 10],
#     'learning_rate': [0.1, 0.01, 0.05],
#     'max_depth': [2, 4, 6, 8, 10]
# }

# grid_search = GridSearchCV(estimator=card_fraud_model_XGB, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
# grid_search.fit(X_train, y_train)

# print(grid_search.best_params_)
# print(grid_search.best_score_)
# print(grid_search.best_estimator_)
# print(grid_search.best_index_)
# print(grid_search.scorer_)
# print(grid_search.n_splits_)
# print(grid_search.refit_time_)
# print(grid_search.cv_results_)
# print(grid_search.predict(X_valid))


# on fit les modeles sur l'entiereté du training set
card_fraud_model.fit(OH_X_train, y)
card_fraud_model_XGB.fit(OH_X_train, y)


c:\Users\qjado\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


0.8717948717948718
[[19089     2]
 [    8    34]]


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=2, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=0, ...)

We now make our predictions

In [223]:
# we make predictions which we will submit to Zindi, it need to be submit as a csv file containing the TransactionId and FraudResult
# The Transaction id is writen as "TransactionId_XXXX" so we need to split it and take the last element
predictions = card_fraud_model_XGB.predict(OH_X_test)
# we print the first 5 predictions
print(predictions[:5])

# On réintègre la colonne TransactionId a partir de X_test_index
OH_X_test['TransactionId'] = X_test_index

# We create a dataframe containing the TransactionId and the predictions
submission = pd.DataFrame({'TransactionId': OH_X_test.TransactionId, 'FraudResult': predictions})
print(submission.head())

# We add 'TransactionId_' to the TransactionId column
submission['TransactionId'] = submission['TransactionId'].astype(str)

# We save the dataframe as a csv file
submission.to_csv('submission.csv', index=False)

submission.head()

[0 0 0 0 0]
         TransactionId  FraudResult
0  TransactionId_50600            0
1  TransactionId_95109            0
2  TransactionId_47357            0
3  TransactionId_28185            0
4  TransactionId_22140            0


,TransactionId,FraudResult
0,TransactionId_50600,0
1,TransactionId_95109,0
2,TransactionId_47357,0
3,TransactionId_28185,0
4,TransactionId_22140,0
